### This is an implementation of an FFT on a song
The inspiration for this implementation came from http://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

In [ ]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct
from scipy.io import wavfile
from scipy.signal import blackman, hanning, hamming
import numpy as np
from scipy import signal

%matplotlib inline

# Location of the song
files_location = "C:/Users/bre49823/Google Drive/dt_16bars_102rap.wav"

#### Read in the data and create a plot of the raw audio file
The raw data is in the time domain and using Fourier transform I'll convert it to frequency spectrum

In [ ]:
# Read in the raw audio data and get the sample rate (in samples/sec)
sample_rate, soundtrack_data = wavfile.read(files_location)
audio_data = soundtrack_data.T[0]                      # this is a two channel soundtrack, get only one of the tracks
audio_data = audio_data[0:int(10 * sample_rate)]      # keep only the first 10 seconds

time = np.arange(0, float(audio_data.shape[0]), 1) / sample_rate

In [ ]:
# Plot the raw signal - amplitude (or loudness) over time
plt.figure(figsize = (30, 20))
plt.plot(time, audio_data, linewidth = 0.1, alpha = 2, color='#ff0000')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Audio signal in the time domain")

#### Pre-emphasis
Apply pre-emphasis to the audio data. This makes the signal cleaner with less noise. It is done to balance the frequency spectrum since high frequencies have lower magnitude than lower frequncies.
<br>
The pre-emphasis filter is y(t) = x(t) - emphasis_coeff * (t - 1)
<br>
Typical values for the emphasis coefficients are 0.95 and 0.97

In [ ]:
# Apply pre-emphasis filter
emphasis_coeff = 0.95
emphasized_audio_data = np.append(audio_data[0], audio_data[1:] - emphasis_coeff * audio_data[:-1])

# Plot the emphasized signal
plt.figure(figsize = (30, 20))
plt.plot(time, emphasized_audio_data, linewidth = 0.1, alpha = 2, color='#ff0000')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Pre-emphaised audio signal in the time domain")

In [ ]:
# Framing



In [ ]:
# Create a window for the FFT


In [ ]:
# Calculate FFT and power spectrum


In [ ]:
# Plot spectrogram
plt.imshow(np.transpose(ims), origin = "lower", aspect = "auto", cmap = colormap, interpolation = "none")


In [ ]:
# Create filter banks
